In [1]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util

# --- Load caption similarity model ---
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Define weights for each category ---
weights = {
    "main_objects": 0.55,
    "main_object_attributes": 0.05,
    "location": 0.15,
    "action": 0.15,
    "surroundings": 0.05,
    "background": 0.05
}

# --- Function to parse structured caption into dictionary ---
def parse_caption(raw_caption: str):
    categories = {
        "main_objects": r"main_objects:\s*(.*?)(?=main_object_attributes:|location:|action:|surroundings:|background:|$)",
        "main_object_attributes": r"main_object_attributes:\s*(.*?)(?=location:|action:|surroundings:|background:|$)",
        "location": r"location:\s*(.*?)(?=action:|surroundings:|background:|$)",
        "action": r"action:\s*(.*?)(?=surroundings:|background:|$)",
        "surroundings": r"surroundings:\s*(.*?)(?=background:|$)",
        "background": r"background:\s*(.*)"
    }
    parsed = {}
    for key, pattern in categories.items():
        match = re.search(pattern, raw_caption, re.IGNORECASE)
        if match:
            items = [i.strip() for i in match.group(1).split(",") if i.strip()]
            parsed[key] = items
        else:
            parsed[key] = []
    return parsed

# --- Function to compute cosine similarity ---
def compute_similarity(list1, list2):
    if not list1 or not list2:
        return 0.0
    text1 = ', '.join(list1)
    text2 = ', '.join(list2)
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    return float(util.cos_sim(embedding1, embedding2).item())

# --- Function to compute total weighted score ---
def score_pair(teacher_caption, student_caption):
    total_score = 0.0
    details = {}
    for key in weights:
        t_val = teacher_caption.get(key, [])
        s_val = student_caption.get(key, [])
        sim = compute_similarity(t_val, s_val)
        weighted = sim * weights[key]
        total_score += weighted
        details[f"{key}_sim"] = round(sim, 3)
        details[f"{key}_weighted"] = round(weighted, 3)
    details["final_score"] = round(total_score, 3)
    return details

# --- Load CSV ---
input_path = "/Users/fatihwolf/Downloads/cleaned_output.csv"  # Replace with your path
df = pd.read_csv(input_path)

# --- Apply scoring ---
results = []

for idx, row in df.iterrows():
    student_text = str(row["Student"])
    teacher_text = str(row["Teacher"])
    parsed_student = parse_caption(student_text)
    parsed_teacher = parse_caption(teacher_text)
    similarity_scores = score_pair(parsed_teacher, parsed_student)

    result_row = {
        "Row": row["Row"],
        "Student": student_text,
        "Teacher": teacher_text,
        **similarity_scores
    }
    results.append(result_row)

# --- Save results to CSV ---
output_path = "/Users/fatihwolf/Downloads/caption_sim_scores_test.csv"
pd.DataFrame(results).to_csv(output_path, index=False)
print(f"✅ Scoring complete! Results saved to {output_path}")


✅ Scoring complete! Results saved to /Users/fatihwolf/Downloads/caption_sim_scores_test.csv


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import torch
import clip
import os
import pandas as pd

# 📁 Set your image folder path
image_folder = "/Users/fatihwolf/Downloads/workspace_backup/50set"

# 🧠 Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

def get_clip_embedding(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        return model.encode_image(image).cpu().numpy()[0]

# 📊 Data collection
results = []

# Loop through image pairs
for i in range(1, 51):  # From 1 to 50
    index = str(i).zfill(3)  # Formats 1 → '001', 2 → '002', ...
    teacher_path = os.path.join(image_folder, f"{index}_teacher.png")
    student_path = os.path.join(image_folder, f"{index}_student.png")

    if not os.path.exists(teacher_path) or not os.path.exists(student_path):
        print(f"⚠️ Skipping row {i}: Missing file(s)")
        continue

    # Load and preprocess images
    img1 = cv2.imread(teacher_path)
    img2 = cv2.imread(student_path)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Resize to match
    height = min(img1_gray.shape[0], img2_gray.shape[0])
    width = min(img1_gray.shape[1], img2_gray.shape[1])
    img1_resized = cv2.resize(img1_gray, (width, height))
    img2_resized = cv2.resize(img2_gray, (width, height))

    # Pixel similarity
    mse_val = np.mean((img1_resized - img2_resized) ** 2)
    ssim_val = ssim(img1_resized, img2_resized)

    # CLIP semantic similarity
    embedding1 = get_clip_embedding(teacher_path)
    embedding2 = get_clip_embedding(student_path)
    cosine_similarity = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    scaled_clip_score = ((cosine_similarity - 0.6) / 0.4) * 100  # Scale to 0–100

    # Append results
    results.append({
        "Row": i,
        "MSE": round(mse_val, 2),
        "SSIM": round(ssim_val, 4),
        "CLIP Cosine": round(cosine_similarity, 4),
        "Scaled CLIP Score (0–100)": round(scaled_clip_score, 2)
    })

# 📝 Save to Excel
df = pd.DataFrame(results)
output_excel = os.path.join(image_folder, "image_clip_scores_test.xlsx")
df.to_excel(output_excel, index=False)

print(f"✅ Saved results to {output_excel}")


✅ Saved results to /Users/fatihwolf/Downloads/workspace_backup/50set/image_clip_scores_test.xlsx
